# Random Boolean Network simulations for Type-1 lesioning (with noise prob. 0.05)

### Paper reference: Kocaoglu and Alexander, Degeneracy measures in biologically plausible random Boolean networks, BMC Bioinformatics.

In [1]:
import numpy as np
import pandas as pd
import string
import random
from itertools import chain

### To quickly test some condition and print stuff, uncomment below cell and the ones in "FUNCTIONS" cell

In [ ]:
# n=100
# k=1
# lesion=0

### FUNCTIONS:

In [3]:
def nodeinits(n):
    node_inits = np.random.choice(a=[1,0], size=(1, n))
    node_inits = np.reshape(node_inits,n)
    
    return node_inits

# node_initials= nodeinits(n)


def presets(node_initials, n, lesion): 

    # target nodes:
    list1= np.arange(n)
    list2= np.arange(n)
    for i in range(n):#no self-connections
        while list1[i]==i:
          np.random.shuffle((list1)) 
        while list2[i]==i:
          np.random.shuffle((list2))
          # print(list1)
    for i in range(n):
        while list1[i]==list2[i]: #don't repeat the same node in connections
          list2[i] = np.random.randint(n)

    adj_list = np.column_stack((list1,list2))
#     print(adj_list)

    # randomly cut some nodes so that some nodes have only one connection:
    chosenlist=np.array([])
    newlist= list(adj_list) 

    howmanyoneconn= np.random.choice(n)

    for i in range(howmanyoneconn):
        chosen= np.random.choice(n)
        while chosen in chosenlist:
            chosen= np.random.choice(n)
        chosenlist = np.append(chosenlist, chosen)
        # print('chosen targets index', chosen)
        nestedlist= list(newlist[chosen])
        pick = np.random.randint(0,2)
        # print('pick which specific index to be deleted',pick)
        nestedlist.remove(nestedlist[pick]) 
        # print('after deletion what is left',nestedlist)
        newlist[chosen] = np.array(nestedlist)
        # print(newlist)

    # newlist = np.asarray(newlist, dtype=object)

    #lesion here:
    lesioned = list(string.ascii_lowercase)
    lesioned = lesioned[0:lesion]
    lesioned = [_ for _ in lesioned] #just for empty list indexing

    if lesion>= 1:
        for lesions in range(lesion):
            # print('list of lesion range',lesions)
            lesionthisgene = np.random.choice(n)
            while lesionthisgene in lesioned:
                lesionthisgene = np.random.choice(n)
            newlist[lesionthisgene] = np.array([lesionthisgene])
            lesioned[lesions] = lesionthisgene
        else:
            pass


    return newlist 

# adj_list= presets(node_initials, n, lesion)

def targs(n, newlist, node_initials):
    # target_values = np.zeros([n,max_conn])
    target_values = [_ for _ in range(n)]

  # target_values = np.array(target_values, dtype=object)


    for i in range(n):
        try:
          x,y =  newlist[i]
          next = node_initials[x], node_initials[y]
          next = np.asarray(next)
        except ValueError:
          x = newlist[i]
          next = node_initials[x]
        target_values[i] = next
      # np.stack(target_values,next)
    # print('next',next,'type',type(next))
    # print(target_values)
    # 

    # target_values = target_values.astype(int)  

    # print(target_values)
    # return adj_list, target_values
    return target_values


# target_values = targs(n, adj_list, node_initials)
  
# adj_list, target_values= presets(n)
# print(newlist)
# print(target_values)

def boolrules(target_values):

  operators= ['a','o','an','on','xor','nan','non','na','no'] # bool operators
  # a = logical_and(x,y)
  # o = logical_or(x,y)
  # n = logical_not(x)
  # an = logical_and(x,logical_not(y))
  # on = logical_or(x,logical_not(y))
  ruleset=[]
  new_values= np.zeros(n)
  for i in range(n):
    operator= np.random.choice(operators,1)
    # print(operator)
    try: 
      tar1,tar2= target_values[i]
      

      if operator=='a':
        new_values[i]= np.logical_and(tar1,tar2)
      elif operator=='an':
        new_values[i]= np.logical_and(tar1,np.logical_not(tar2))
      elif operator=='o':
        new_values[i]= np.logical_or(tar1,tar2)
      elif operator=='on':
        new_values[i]= np.logical_or(tar1,np.logical_not(tar2))
      elif operator=='xor':
        new_values[i]= np.logical_xor(tar1,tar2)
      elif operator == 'nan':
        new_values[i]= np.logical_and(np.logical_not(tar1),np.logical_not(tar2))
      elif operator == 'non':
        new_values[i]= np.logical_or(np.logical_not(tar1),np.logical_not(tar2))
      elif operator=='na':
        new_values[i]= np.logical_and(np.logical_not(tar1),tar2)
      elif operator=='no':
        new_values[i]= np.logical_or(np.logical_not(tar1),tar2)

      
      ruleset += [operator.tolist()]

    except ValueError:
      tar1= target_values[i]
      if np.random.choice([0,1], p=[0.5, 0.5])>=0.5:
        new_values[i] = np.logical_not(tar1)
        operator = ['not']

      else:
        new_values[i] = tar1
        operator = ['copy']

      ruleset += [operator]

#     print(tar1,tar2)


  new_values= new_values.astype(int)
  return new_values, ruleset

# new_values, ruleset = boolrules(target_values)

# print(new_values)
# print(ruleset)

def updatestates(new_values,ruleset,adj_list):
  new_values= new_values.astype(int)
  updated_values = np.zeros(np.shape(new_values))
  # print(new_values)
  # print(ruleset)
  # print(adj_list)

  for i in range(n):
    try:
      x,y =  adj_list[i]
      next = new_values[x], new_values[y]
      next = np.asarray(next)
    except ValueError:
      x = adj_list[i]
      next = new_values[x]

    rule= ruleset[i]
    if rule==['a']:
      updated_values[i]= np.logical_and(x,y)
      # print('hello this is AND')
    elif rule==['an']:
      updated_values[i]= np.logical_and(x,np.logical_not(y))
    elif rule==['o']:
      updated_values[i]= np.logical_or(x,y)
    elif rule==['on']:
      updated_values[i]= np.logical_or(x,np.logical_not(y))
    elif rule==['xor']:
      updated_values[i] = np.logical_xor(x,y)
    elif rule==['nan']:
      updated_values[i] = np.logical_and(np.logical_not(x),np.logical_not(y))
    elif rule==['non']:
      updated_values[i] = np.logical_or(np.logical_not(x),np.logical_not(y))
    elif rule==['na']:
      updated_values[i] = np.logical_and(np.logical_not(x),y)
    elif rule==['no']:
      updated_values[i] = np.logical_or(np.logical_not(x),y)

    elif rule==['copy']:
      updated_values[i]= new_values[x]
    elif rule==['not']:
      updated_values[i]= np.logical_not(x)

  
  updated_values = updated_values.astype(int)
  return updated_values 

# updated_values = updatestates(new_values,ruleset,adj_list)
# updated_values = updatestates(new_values,ruleset,newlist)

# print(updated_values)



def pert_nodes(k):

  pertb = [_ for _ in range(n)]
  perts = pertb[0:k]
  return perts



### Simulations start below

In [ ]:
networks = 1000 # How many networks will be generated for each perturbation condition in a given lesoning parameter
n = 10 # the number of genes

experiments = 10
period = 100
timepoints = experiments * period
burnin = experiments #burn-in at the beginning of each experiment throughout the simulation


for k in range(1,n+1):
    for lesion in range(n): # here it eventually lesions networks n genes; 
        # but, for example, if put "1", it will not do any lesioning etc.
        for network in range(networks):

            node_inits= nodeinits(n)
            genelist = presets(node_inits, n, lesion)


            for experiment in range(experiments+1):

                target_vals = targs(n, genelist, node_inits)
                node_inits= nodeinits(n) #change inits for next round of experiments
                new_vals, theruleset = boolrules(target_vals)
                perturbedlist = pert_nodes(k) # perturbed gene list

                if experiment > 0:
                    makedf = pd.DataFrame(expdata_each)
                    allexpdata = pd.concat([allexpdata, makedf], axis=1)
                elif experiment == 0:
                    allexpdata = pd.DataFrame({'A' : []})

                expdata = np.zeros((period, n)) #initialize to concat later 

                for i in range(period): 
                    
                    expvector = updatestates(new_vals,theruleset,genelist)
                    new_vals = expvector # update the states by feeding prev. here
                    # print('exp vector',new_vals)

                    for gene in range(k):
                        per = perturbedlist[gene]
                      # print('perturbed node', per)  

                      #weighted coin flip for deciding perturbing/flipping the state of the node(s)
                        coin= ['heads','tails']
                        flip = np.random.choice(coin, p=[0.25, 0.75]) #before 0.1, 0.9
                        if flip=='heads':
                            new_vals[per] = np.logical_not(expvector[per])
                        # print('after flip vector', new_vals)
                        else:
                            pass #not flip


                    # ADD SMALL FLIPPY THING FOR ALL GENES - INCLUDING PERTURBED ONES
                    for allgenes in range(n):
                        coin= ['heads','tails']
                        flip = np.random.choice(coin, p=[0.05, 0.95])
                        
                        if flip=='heads':
                            new_vals[allgenes] = np.logical_not(expvector[allgenes])
                        else:
                            pass

                        
                    expdata[i]= expvector
#                     print(expdata[i])
                    # print(np.shape(expdata[i]))
                    expdata_each= np.asarray(expdata.T.astype(int))
#                     print('before',expdata_each.shape)


                    #BURN-IN
                    expdata_each = np.delete(expdata_each, 0, 1)
#                     print('after',expdata_each.shape)
#                     print(expdata_each)


            #AFTER EXPERIMENTS
            expdatafinal = allexpdata # conc one
            genenames = list([f'G{genenum}' for genenum in range(1,n+1)])
            colnameslist = [[f'Exp{expt}Point{t}' for t in range(1,period)] for expt in range(experiments)] #burnin skipping by range(1,period)
#             print(colnameslist)
            colnameslist = list(chain.from_iterable(colnameslist))
            df = pd.DataFrame(expdatafinal)
            df.index = genenames
            dataset = df.drop(columns=['A'])
            dataset.columns = colnameslist
            datafinal = f'~/Desktop/some_folder/Type1/ExpressionData_k{k}_lesionedgenes{lesion}_net{network}.csv'
            # print(datafinal)
#             df.to_csv(datafinal)
            dataset.to_csv(datafinal)
#             print(dataset)